There was some problem in building the past .ttl file. I'll redo the steps and debug.

Now that the entities are on Wikidata, while there is no has_positive_marker property there, we can make a local RDF file using Wikidata IDs. 

In [7]:
import pandas as pd 

gene_reference = pd.read_csv("../results/human_gene_reference_from_panglao_to_wikidata_04_11_2020.csv")

cell_type_reference = pd.read_csv("../results/cell_type_reference_from_panglao_to_wikidata_31_10_2020.csv")

markers = pd.read_csv("../data/PanglaoDB_markers_27_Mar_2020.tsv", sep="\t")

We want, though, the species specific cell types. Let's get them. 

In [8]:
from wikidata2df import wikidata2df

# A SPARQL query to return all cats in Wikidata!

query = """
SELECT ?item ?itemLabel ?superclass
WHERE
{
?item wdt:P31 wd:Q189118. 
?item wdt:P361 ?superclass. 
?superclass  wdt:P31 wd:Q189118. # cell type
?item wdt:P703 wd:Q15978631. # human cell type
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

dataframe_to_join = wikidata2df(query) 

In [12]:
dataframe_to_join

,superclass,item,itemLabel
0,Q735343,Q101405096,human müller cell
1,Q1467330,Q101404949,human myofibroblast
2,Q1956694,Q101404940,human myoblast
3,Q4381253,Q101404941,human stromal cell
4,Q4532277,Q101404922,human epsilon cell
5,Q5249153,Q101404881,human decidual cell
6,Q5297290,Q101405098,human dopaminergic neuron
7,Q10599959,Q101405206,human neuroendocrine cell
8,Q28646122,Q101405078,human ependymal cell
9,Q66508979,Q101404901,human smooth muscle cell


The query syntax is wrong! It is looking for "part of" while it should be looking for "subclass of"

I'll fix the 9 items and re-do the query. 

In [1]:
from wikidata2df import wikidata2df

# A SPARQL query to return all cats in Wikidata!

query = """
SELECT ?item ?itemLabel ?superclass
WHERE
{
?item wdt:P31 wd:Q189118. 
?item wdt:P279 ?superclass. 
?superclass  wdt:P31 wd:Q189118. # cell type
?item wdt:P703 wd:Q15978631. # human cell type
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

dataframe_to_join = wikidata2df(query)

In [2]:
dataframe_to_join

,superclass,itemLabel,item
0,Q502961,human astrocyte,Q67801129
1,Q71292398,Human MTG Oligodendrocyte,Q71307001
2,Q71292660,Human MTG Microglial cell,Q71307005
3,Q71292831,human cerebral cortex GABAergic interneuron,Q71299560
4,Q71299560,Human cerebral cortex MTG GABAergic interneuron,Q71307007
...,...,...,...
283,Q101062629,human red pulp macrophages,Q101404936
284,Q101062659,human salivary mucous cell,Q101404929
285,Q101062667,human ionocyte,Q101404947
286,Q101062687,human noradrenergic neuron,Q101404932


In [3]:
cell_type_reference

NameError: name 'cell_type_reference' is not defined

In [14]:
cell_type_reference.merge(dataframe_to_join, left_on="wikidata", right_on="superclass")

,Unnamed: 0,panglao,wikidata,superclass,item,itemLabel
0,0,Basal cells,Q101062513,Q101062513,Q101404853,human basal cell
1,1,Trigeminal neurons,Q101062590,Q101062590,Q101404857,human trigeminal neuron
2,2,Juxtaglomerular cells,Q2596226,Q2596226,Q101404858,human juxtaglomerular cell
3,4,Fibroblasts,Q463418,Q463418,Q101404861,human fibroblast
4,5,Motor neurons,Q1133903,Q1133903,Q101404862,human motor neuron
...,...,...,...,...,...,...
106,176,Enterochromaffin cells,Q568290,Q568290,Q101405125,human enterochromaffin cell
107,177,Neuroendocrine cells,Q10599959,Q10599959,Q101405206,human neuroendocrine cell
108,180,Spermatocyte and Spermatozoa,Q101003191,Q101003191,Q101405207,human spermatocyte and spermatozoa
109,184,Cortical interneurons,Q101004030,Q101004030,Q101405214,human cortical interneuron


Some rows are still being lost in the process. Let's understand why. 

Row 3 was lost: Pancreatic stellate cells	Q1164962

It is present in wikidata, but without a "P279" statement to the general cell type. 

Lets look for other cases. 



In [26]:
import numpy as np


main_list = np.setdiff1d(cell_type_reference["wikidata"].dropna(), dataframe_to_join["superclass"].dropna())
cell_type_reference[[a in main_list for a in cell_type_reference["wikidata"]]]

,Unnamed: 0,panglao,wikidata
3,3,Pancreatic stellate cells,Q1164962
7,7,Parietal cells,Q1640093
8,8,Peri-islet Schwann cells,Q101062584
10,10,Dendritic cells,Q506253
11,11,Tuft cells,Q25325383
...,...,...,...
178,178,Astrocyte progenitor cells,Q101001053
179,179,Multinuclear myocytes,Q101003165
181,181,Granulocytes,Q223143
182,182,Theca cells,Q101003314


I will re-run the Quickstatements used for adding the P279 statesments originally and hope for a magic fix. 

(../quickstatements/P279_correction_08_11_2020.qs)


In [1]:
from wikidata2df import wikidata2df

# A SPARQL query to return all cats in Wikidata!

query = """
SELECT ?item ?itemLabel ?superclass
WHERE
{
?item wdt:P31 wd:Q189118. 
?item wdt:P279 ?superclass. 
?superclass  wdt:P31 wd:Q189118. # cell type
?item wdt:P703 wd:Q15978631. # human cell type
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

dataframe_to_join = wikidata2df(query)


In [2]:
dataframe_to_join

,superclass,itemLabel,item
0,Q502961,human astrocyte,Q67801129
1,Q71292398,Human MTG Oligodendrocyte,Q71307001
2,Q71292660,Human MTG Microglial cell,Q71307005
3,Q71292831,human cerebral cortex GABAergic interneuron,Q71299560
4,Q71299560,Human cerebral cortex MTG GABAergic interneuron,Q71307007
...,...,...,...
283,Q101062629,human red pulp macrophages,Q101404936
284,Q101062659,human salivary mucous cell,Q101404929
285,Q101062667,human ionocyte,Q101404947
286,Q101062687,human noradrenergic neuron,Q101404932


I had to restart the kernel so to clear the cache of the query. 

In [3]:
import pandas as pd 

gene_reference = pd.read_csv("../results/human_gene_reference_from_panglao_to_wikidata_04_11_2020.csv")

cell_type_reference = pd.read_csv("../results/cell_type_reference_from_panglao_to_wikidata_31_10_2020.csv")

markers = pd.read_csv("../data/PanglaoDB_markers_27_Mar_2020.tsv", sep="\t")

In [4]:
cell_type_reference.merge(dataframe_to_join, left_on="wikidata", right_on="superclass")

,Unnamed: 0,panglao,wikidata,superclass,itemLabel,item
0,0,Basal cells,Q101062513,Q101062513,human basal cell,Q101404853
1,1,Trigeminal neurons,Q101062590,Q101062590,human trigeminal neuron,Q101404857
2,2,Juxtaglomerular cells,Q2596226,Q2596226,human juxtaglomerular cell,Q101404858
3,3,Pancreatic stellate cells,Q1164962,Q1164962,human pancreatic stellate cell,Q101404859
4,4,Fibroblasts,Q463418,Q463418,human fibroblast,Q101404861
...,...,...,...,...,...,...
183,181,Granulocytes,Q223143,Q223143,human granulocyte,Q101405212
184,182,Theca cells,Q101003314,Q101003314,human theca cell,Q101405210
185,184,Cortical interneurons,Q101004030,Q101004030,human cortical interneuron,Q101405214
186,186,Peripheral blood mononuclear cells,Q736033,Q736033,human peripheral blood mononuclear cell,Q101405209


In [5]:
import numpy as np


main_list = np.setdiff1d(cell_type_reference["wikidata"], dataframe_to_join["superclass"].dropna())
cell_type_reference[[a in main_list for a in cell_type_reference["wikidata"]]]

,Unnamed: 0,panglao,wikidata
7,7,Parietal cells,Q1640093
21,21,Transient cells,EXCLUDE
40,40,Serotonergic neurons,Q70220966
53,53,Bergmann glia,Q101062516
82,82,Satellite glial cells,Q63710
113,113,Foveolar cells,Q66592466
160,160,Acinar cells,Q66590269
162,162,Clara cells,Q645712
168,168,Glomus cells,Q4778753


I am fixing some of those manually. 
Satellite glial cells	Q63710 has a typo, it should point to https://www.wikidata.org/wiki/Q637104

The query seems to be buggy, though. 
I'll proceed and note down the problems. 


In [7]:
cell_type_reference = cell_type_reference.merge(dataframe_to_join, left_on="wikidata", right_on="superclass")

In [8]:
cell_type_reference.to_csv("../results/human_cell_type_reference_21_11_2020.csv")
cell_type_reference.head()

,Unnamed: 0,panglao,wikidata,superclass,itemLabel,item
0,0,Basal cells,Q101062513,Q101062513,human basal cell,Q101404853
1,1,Trigeminal neurons,Q101062590,Q101062590,human trigeminal neuron,Q101404857
2,2,Juxtaglomerular cells,Q2596226,Q2596226,human juxtaglomerular cell,Q101404858
3,3,Pancreatic stellate cells,Q1164962,Q1164962,human pancreatic stellate cell,Q101404859
4,4,Fibroblasts,Q463418,Q463418,human fibroblast,Q101404861


In [9]:
human_markers = markers[["Hs" in val for val in markers["species"]]]

In [10]:
human_markers_lean = human_markers[["official gene symbol", "cell type"]]
human_markers_lean.to_csv("../results/human_markers_reconciled_13_11_2020.csv")
human_markers_lean.head()

,official gene symbol,cell type
0,CTRB1,Acinar cells
1,KLK1,Acinar cells
2,RBPJL,Acinar cells
3,PTF1A,Acinar cells
5,CELA3A,Acinar cells


In [11]:
human_markers_lean = human_markers_lean.merge(cell_type_reference, left_on="cell type", right_on="panglao")[["official gene symbol", "cell type", "item"]]

human_markers_lean.columns = ["official gene symbol", "cell type", "cell type id"]

In [12]:
human_markers_lean = human_markers_lean.merge(gene_reference, left_on="official gene symbol", right_on="panglao")[["official gene symbol", "cell type", "cell type id", "wikidata"]]

human_markers_lean.columns = ["official gene symbol", "cell type", "cell type id", "gene id"]
human_markers_lean

,official gene symbol,cell type,cell type id,gene id
0,CEBPA,Adipocyte progenitor cells,Q101404942,Q17861031
1,CEBPA,Adipocyte progenitor cells,Q101404943,Q17861031
2,CEBPA,Basophils,Q101405089,Q17861031
3,CEBPA,Hepatoblasts,Q101404910,Q17861031
4,CEBPA,Hepatocytes,Q101405101,Q17861031
...,...,...,...,...
7896,SLC14A1,Urothelial cells,Q101404883,Q18031563
7897,UPK3A,Urothelial cells,Q101404883,Q18032295
7898,UPK1A,Urothelial cells,Q101404883,Q18036055
7899,UPK2,Urothelial cells,Q101404883,Q18032294


In [13]:
import rdflib

g = rdflib.Graph()

for index, row in human_markers_lean.iterrows():

    ctp = rdflib.Namespace("http://celltypes.wiki.opencura.com/entity/")
    wd = rdflib.Namespace("http://www.wikidata.org/entity/")
    wdt = rdflib.Namespace("http://www.wikidata.org/prop/direct/")

    s = rdflib.term.URIRef(wd+row["cell type id"])
    p1 = rdflib.term.URIRef(ctp+"P9")
    o1 = rdflib.term.URIRef(wd+row["gene id"])
    
    g.bind("ctp", ctp)
    g.bind("wd", wd)
    g.bind("wdt", wdt)
    g.add((s, p1, o1))

g.serialize(destination='../results/human_cell_type_markers_21_11_2020.ttl', format='turtle')

There we go. Now we have an _almost_ complete .ttl file.